In [26]:
import os, sys, time, datetime, random

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

from utils import *

In [27]:
ROOT_DIR = os.getcwd()
DATA_ROOT = os.path.join(ROOT_DIR, 'data')
TRAIN_PATH = os.path.join(DATA_ROOT, 'train')
TEST_PATH = os.path.join(DATA_ROOT, 'test')

train_df = pd.read_csv(os.path.join(TRAIN_PATH, 'train_output.csv'))
test_df = pd.read_csv(os.path.join(TEST_PATH, 'test_output.csv'))

In [28]:
transform = get_transform()

In [29]:
train_dset = ObjDetectionDataset(TRAIN_PATH, train_df, transform=get_transform())
test_dset = ObjDetectionDataset(TEST_PATH, test_df, transform=get_transform())

len(train_dset), len(test_dset)

(1204, 300)

In [30]:
train_loader = DataLoader(train_dset, batch_size=1, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dset, batch_size=1, shuffle=False, collate_fn=collate_fn)

len(train_loader), len(test_loader)

NameError: name 'collate_fn' is not defined

In [31]:
unique_labels = set()
for labels in train_dset.df['label_idx']:
    unique_labels = unique_labels.union(set(labels))
for labels in test_dset.df['label_idx']:
    unique_labels = unique_labels.union(set(labels))

num_classes = len(unique_labels)
num_classes

4

In [214]:
config_path = os.path.join(ROOT_DIR, 'config', 'yolov3.cfg')
weights_path = os.path.join(ROOT_DIR, 'config', 'yolov3.weights')
img_size = 416
conf_thres = 0.8
nms_thres = 0.4
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Darknet(config_path, img_size=img_size)
model.load_weights(weights_path)
model.to(device)

model

Darknet(
  (module_list): ModuleList(
    (0): Sequential(
      (conv_0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm_0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_0): LeakyReLU(negative_slope=0.1)
    )
    (1): Sequential(
      (conv_1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (batch_norm_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_1): LeakyReLU(negative_slope=0.1)
    )
    (2): Sequential(
      (conv_2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (batch_norm_2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_2): LeakyReLU(negative_slope=0.1)
    )
    (3): Sequential(
      (conv_3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm_3): BatchNorm2d(64, eps=1e-05, momen

In [213]:
images, labels = next(iter(train_loader))
images = torch.stack([image.to(device) for image in images], dim=0)
labels = labels[0].view(1, *labels[0].size())

outputs = model(images)
outputs

RuntimeError: shape '[1, 3, 9, 13, 13]' is invalid for input of size 43095

In [206]:
utils.non_max_suppression(outputs, 80)

[tensor([[-173.3735, -182.2556,  224.9124,  213.7543,    0.9428,    0.5943,
            20.0000]])]

In [195]:
labels

tensor([[[  3.0000, 251.0627, 214.7459, 263.5751, 327.1782]]],
       dtype=torch.float64)